In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt

In [2]:
df_all = pd.read_csv("./data/sfpd_dispatch_data_subset.csv")
num_rows = df_all.shape[0]
df_all["received_timestamp"] = pd.to_datetime(df_all["received_timestamp"])

In [3]:
counter_nan = df_all.isnull().sum()
counter_no_nan = counter_nan[counter_nan==0]

In [4]:
df_cleaned = df_all[counter_no_nan.keys()]
kept_keys = counter_no_nan.keys().values
kept_keys

array(['call_number', 'unit_id', 'incident_number', 'call_type',
       'call_date', 'watch_date', 'received_timestamp', 'entry_timestamp',
       'dispatch_timestamp', 'call_final_disposition',
       'available_timestamp', 'address', 'zipcode_of_incident',
       'battalion', 'station_area', 'box', 'original_priority',
       'priority', 'final_priority', 'als_unit', 'number_of_alarms',
       'unit_type', 'unit_sequence_in_call_dispatch',
       'fire_prevention_district', 'supervisor_district', 'location',
       'row_id', 'latitude', 'longitude', 'x', 'y', 'z'], dtype=object)

In [5]:
# kept_keys = ['unit_id', 'call_type', 'received_timestamp', 'entry_timestamp',
#        'dispatch_timestamp', 'call_final_disposition',
#        'battalion', 'station_area', 'original_priority',
#        'priority', 'final_priority', 'als_unit', 'number_of_alarms', 'x', 'y', 'z', 'unit_type']
kept_keys = ['received_timestamp','latitude', 'longitude', 'unit_type']
df_cleaned = df_cleaned[kept_keys]
df_cleaned.keys()
#zipcode_of_incident

Index(['received_timestamp', 'latitude', 'longitude', 'unit_type'], dtype='object')

In [22]:
#extract hour, day, dotw from timestamp
columns_to_extract = ["received_timestamp", "entry_timestamp", "dispatch_timestamp"]
columns_to_extract = ["received_timestamp"]
full_df = pd.DataFrame()
for c in columns_to_extract:
    new_data = []
    for timestamp in df_cleaned[c]:
        keys = ["date_"+c[:3], "hour_"+c[:3], "dow_"+c[:3]]
        new_data.append({keys[0]: timestamp.day, keys[1]: timestamp.hour, keys[2]: timestamp.dayofweek});
    new_df = pd.DataFrame(new_data)
    full_df = new_df.join(full_df, rsuffix='_next')
    df_cleaned = df_cleaned.drop(c, axis=1)
df_cleaned = df_cleaned.join(full_df, rsuffix="_next")

KeyError: 'received_timestamp'

In [8]:
call_type_mapping = {
    'Alarms': 0,
    'Citizen Assist / Service Call': 1,
    'Electrical Hazard': 2,
    'Elevator / Escalator Rescue': 3,
    'Fuel Spill': 4,
    'Gas Leak (Natural and LP Gases)': 5,
    'HazMat': 6,
    'Medical Incident': 7,
    'Odor (Strange / Unknown)': 8,
    'Other': 9,
    'Outside Fire': 10,
    'Smoke Investigation (Outside)': 11,
    'Structure Fire': 12,
    'Traffic Collision': 13,
    'Train / Rail Incident': 14,
    'Vehicle Fire': 15,
    'Water Rescue': 16    
}
unit_type_mapping = {
    'CHIEF': 0,
    'ENGINE': 1,
    'INVESTIGATION': 2,
    'MEDIC': 3,
    'PRIVATE': 4,
    'RESCUE CAPTAIN': 5,
    'RESCUE SQUAD': 6,
    'SUPPORT': 7,
    'TRUCK': 8
}
fdisp = {
    'Against Medical Advice': 0,
    'Cancelled': 1,
    'Code 2 Transport': 2,
    'Code 3 Transport': 3,
    'Fire': 4,
    'Gone on Arrival': 5,
    'Medical Examiner': 6,
    'No Merit': 7,
    'Other': 8,
    'Patient Declined Transport': 9,
    'SFPD': 10,
    'Unable to Locate': 11
}
boolean_map = {
    'False': 0,
    'True': 1
}
battalion_mapping = {
    'B01': 0,
    'B02': 1,
    'B03': 2,
    'B04': 3,
    'B05': 4,
    'B06': 5,
    'B07': 6,
    'B08': 7,
    'B09': 8,
    'B10': 9,
    'B99': 10
}
#df_cleaned.groupby('zipcode_of_incident')['zipcode_of_incident'].nunique()
#df_cleaned.groupby('battalion')['battalion'].nunique()

In [23]:
df_cleaned = df_cleaned.replace({'unit_type': unit_type_mapping})
y_data = df_cleaned['unit_type']
df_cleaned = df_cleaned.drop('unit_type', axis=1)

KeyError: 'unit_type'

In [ ]:
# df_onehot = df_cleaned
# onehots = ['unit_id', 'call_type', 'call_final_disposition', 'battalion', 'station_area']
# for i in onehots:
#     onehot = pd.get_dummies(df_onehot[i])
#     df_onehot = df_onehot.drop(i, axis=1)
#     df_onehot = df_onehot.join(onehot, rsuffix='_right')

In [24]:
#df_all["received_timestamp"] = pd.to_datetime(df_all["received_timestamp"])

In [10]:
from sklearn.preprocessing import StandardScaler
x_data = df_cleaned.loc[:,:].values
sscaler = StandardScaler()
x_std= sscaler.fit_transform(x_data)

In [11]:
x_std

array([[ 0.21363192, -2.99211745,  1.59124104, -0.40871212,  0.6565408 ],
       [ 0.1991916 ,  0.17913591,  1.59124104, -0.40871212, -0.93110895],
       [-0.58607219, -1.90084539,  1.59124104, -0.40871212, -1.72493383],
       ...,
       [ 0.6197929 ,  0.64310293, -1.59994427,  0.99226635,  0.18024588],
       [ 1.49158254, -0.30607696, -1.59994427,  0.99226635,  0.49777583],
       [ 0.67706664,  0.38077863, -1.59994427,  0.99226635,  0.0214809 ]])

In [ ]:
# from sklearn.manifold import TSNE
# tsne = TSNE(n_components=3)
# x_std_sm = x_std[:200]
# y_data_sm = y_data[:200]

In [ ]:
# x_2d = tsne.fit_transform(x_std_sm)

In [ ]:
# from mpl_toolkits.mplot3d import Axes3D
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# markers = ('s', 'd', 'o', '^', 'v','<','>','1','2')
# color_map = {0:'red',1:'black',2:'blue',3:'purple',4:'cyan',5:'lightgreen',6:'pink',7:'orange',8:'lightblue'}
# for idx, val in enumerate(x_2d):
#     ax.scatter(val[0],val[1],val[2],c=color_map[y_data_sm[idx]],label=cl)

In [ ]:
# markers = ('s', 'd', 'o', '^', 'v','<','>','1','2')
# color_map = {0:'red',1:'black',2:'blue',3:'purple',4:'cyan',5:'lightgreen',6:'pink',7:'orange',8:'lightblue'}
# plt.figure()
# for idx, val in enumerate(x_2d):
#     plt.scatter(x=val[0],y=val[1],c=color_map[y_data_sm[idx]],label=val)

In [ ]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=3)
# X_pca = pca.fit_transform(x_std[:1000])

In [ ]:
# from mpl_toolkits.mplot3d import Axes3D
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# markers = ('s', 'd', 'o', '^', 'v','<','>','1','2')
# color_map = {0:'red',1:'black',2:'blue',3:'purple',4:'cyan',5:'lightgreen',6:'pink',7:'orange',8:'lightblue'}
# for idx, val in enumerate(X_pca):
#     ax.scatter(val[0],val[1],val[2],c=color_map[y_data[:1000][idx]],label=cl)

In [12]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data)

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [13]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 15, 15, 5))
clf.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 15, 15, 5), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [14]:
predictions = clf.predict(x_test)

In [15]:
from sklearn.metrics import accuracy_score
accuracy_score(predictions, y_test)

0.3584

In [16]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=50)
clf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [17]:
predictions = clf.predict(x_test)

In [18]:
accuracy_score(predictions, y_test)

0.16

In [19]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier()
clf.fit(x_train,y_train)

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [20]:
predictions = clf.predict(x_test)
accuracy_score(predictions, y_test)

0.3128